In [ ]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 3.6 MB/s 
     |████████████████████████████████| 4.0 MB 9.8 MB/s 
     |████████████████████████████████| 1.2 MB 19.8 MB/s 
     |████████████████████████████████| 77 kB 2.7 MB/s 
     |████████████████████████████████| 895 kB 53.6 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=902d7131bcdcf80bd3d383f06233ed48413235846b28ea5fcc96fbcff4000732
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df_training = pd.read_csv("/content/drive/MyDrive/LPJT/public_data_bathurst_cleaned.csv", encoding='unicode_escape')
df_test = pd.read_csv('/content/drive/MyDrive/LPJT/input_data.csv')

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
encoded_labels = labelEncoder.fit_transform(df_training['label'])

In [ ]:
X = pd.concat([df_training['message'].rename('text'), pd.Series(encoded_labels, name='labels')], axis=1)
X.head()

,text,labels
0,hi i want change my address from my credit card,38
1,i need 4 fruit maple oatmeal 3 cold brew froze...,22
2,i wish to travel next month domestic airway,0
3,i need reimbursement my expenses,11
4,i need a copy of insurance for my car,15


In [ ]:
y = X['labels'].values.astype('float32')

In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.1 MB/s 


In [ ]:
X_messages = X['text']

In [ ]:
!pip install unidecode

# remove weird characters from messages
import unidecode
X_messages = X_messages.apply(lambda x: unidecode.unidecode(x))

In [ ]:
# remove numbers from messages
import re
X_messages = X_messages.apply(lambda x: re.sub(r'\d+', '', x))

In [ ]:
# tokenize messages
nltk.download('punkt')
X_messages_tokens = X_messages.apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# remove stopwords
nltk.download('stopwords')

X_messages_tokens = X_messages_tokens.apply(lambda x: [word for word in x if word not in stopwords.words('english')])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# lemmatize words
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
X_messages = X_messages_tokens.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
X_messages

0                      hi want change address credit card
1        need fruit maple oatmeal cold brew frozen coffee
2                  wish travel next month domestic airway
3                              need reimbursement expense
4                                 need copy insurance car
                               ...                       
11618        boarding pas sent email address phone number
11619                                  need ticker bombay
11620                              want musical equipment
11621                                    need window seat
11622                       please money transfer account
Name: text, Length: 11623, dtype: object

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
import unidecode
import re

# Preprocessing functions
def remove_unwanted_chars(df):
    return df.apply(lambda x: unidecode.unidecode(x))

def remove_numbers(df):
    return df.apply(lambda x: re.sub(r'\d+', '', x))

def tokenize_message(df):
    return df.apply(lambda x: word_tokenize(x))

def remove_stopwords(df):
    return df.apply(lambda x: [word for word in x if word not in stopwords.words('english')])

def lemmatize_message(df):
    lemmatizer = WordNetLemmatizer()
    return df.apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x]))

In [ ]:
# Make FunctionTransformers for custom preprocessing functions
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

preprocess_pipe = Pipeline([('removeunwanted', FunctionTransformer(remove_unwanted_chars)),
                            ('removenumbers', FunctionTransformer(remove_numbers)),
                            ('tokenize', FunctionTransformer(tokenize_message)),
                            ('removestopwords', FunctionTransformer(remove_stopwords)),
                            ('lemmatize', FunctionTransformer(lemmatize_message))])

X_train_preprocessed = preprocess_pipe.fit_transform(X['text'])

In [ ]:
X_train_preprocessed

0                      hi want change address credit card
1        need fruit maple oatmeal cold brew frozen coffee
2                  wish travel next month domestic airway
3                              need reimbursement expense
4                                 need copy insurance car
                               ...                       
11618        boarding pas sent email address phone number
11619                                  need ticker bombay
11620                              want musical equipment
11621                                    need window seat
11622                       please money transfer account
Name: text, Length: 11623, dtype: object

In [ ]:
X['text']

0          hi i want change my address from my credit card
1        i need 4 fruit maple oatmeal 3 cold brew froze...
2              i wish to travel next month domestic airway
3                         i need reimbursement my expenses
4                    i need a copy of insurance for my car
                               ...                        
11618    boarding pass to be sent to your email address...
11619                             i need ticker for bombay
11620                             i want musical equipment
11621                                   i need window seat
11622                please money transfer from my account
Name: text, Length: 11623, dtype: object

In [ ]:
X['numwords'] = X['text'].apply(lambda x: len(x.split()))
X.head()

,text,labels,numwords
0,hi i want change my address from my credit card,38,10
1,i need 4 fruit maple oatmeal 3 cold brew froze...,22,11
2,i wish to travel next month domestic airway,0,8
3,i need reimbursement my expenses,11,5
4,i need a copy of insurance for my car,15,9


In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')

from nltk import word_tokenize
from collections import Counter

def count_pos(sentence, pos):
  tokens = nltk.word_tokenize(sentence.lower())
  text = nltk.Text(tokens)
  tags = nltk.pos_tag(text)
  
  counts = Counter(tag for word,tag in tags)
  
  return counts[pos]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
X['numnouns'] = X['text'].apply(count_pos, pos='NN')
X['numverbs'] = X['text'].apply(count_pos, pos='VB')
X.head()

,text,labels,numwords,numnouns,numverbs
0,hi i want change my address from my credit card,38,10,6,0
1,i need 4 fruit maple oatmeal 3 cold brew froze...,22,11,4,0
2,i wish to travel next month domestic airway,0,8,2,1
3,i need reimbursement my expenses,11,5,0,0
4,i need a copy of insurance for my car,15,9,3,0


In [ ]:
from sentence_transformers import SentenceTransformer, LoggingHandler

embedder = SentenceTransformer(model_name_or_path="all-mpnet-base-v2")
X_embeddings = embedder.encode(X_train_preprocessed, convert_to_numpy=True)
X_embeddings

array([[ 0.01708131,  0.00429738, -0.00949069, ..., -0.00591535,
        -0.03251501, -0.02012291],
       [ 0.03394511,  0.00526158, -0.00652598, ...,  0.00122312,
        -0.0452776 , -0.00291192],
       [-0.01984095,  0.03872086, -0.06238953, ...,  0.00311904,
        -0.0303625 , -0.06571841],
       ...,
       [ 0.01027327,  0.00603517,  0.0131581 , ..., -0.03300791,
        -0.02640153, -0.04824821],
       [ 0.01849916,  0.08243959,  0.01891929, ..., -0.00298032,
        -0.0524097 , -0.05037639],
       [ 0.00856035,  0.00793542, -0.03947367, ...,  0.01754491,
        -0.00879176,  0.0044192 ]], dtype=float32)

In [ ]:
X_embeddings.shape

(11623, 768)

## Split Data

In [ ]:
# without metadata
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_embeddings, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)

(9298, 768)
(2325, 768)


In [ ]:
import numpy as np
X_embeddings_metadata = np.concatenate((X_embeddings, np.array(X[['numwords', 'numnouns', 'numverbs']].values)), axis=1)

In [ ]:
# with metadata
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_embeddings_metadata, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)

(9298, 771)
(2325, 771)


## Train Model

In [ ]:
from lightgbm import LGBMClassifier
lgbm_clf = LGBMClassifier(random_state=42)
lgbm_clf.fit(X_train, y_train)

LGBMClassifier(random_state=42)

## Cross-validation

In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score

lgbm_clf = LGBMClassifier(random_state=42)
ami_scores = cross_val_score(lgbm_clf, X_train, y_train, scoring='adjusted_mutual_info_score', cv=5)

# Calculate mean and standard deviation of scores
avg_ami = ami_scores.mean()
stddev_ami = ami_scores.std()

# Print results
print("Scores:", [round(score, 4) for score in ami_scores])
print(f"Mean score: {round(avg_ami, 4)}")
print(f"+/-2 std. dev. range within mean: ({avg_ami - 2*stddev_ami:.4f}, {avg_ami + 2*stddev_ami:.4f})")

## Evaluate model

In [ ]:
# lgbm_clf.fit(X_train, y_train)
y_val_pred = lgbm_clf.predict(X_val)
y_pred_original = labelEncoder.inverse_transform(y_val_pred.astype('int32'))

In [ ]:
y_val_original = labelEncoder.inverse_transform(y_val.astype('int32'))

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

print(confusion_matrix(y_val_original, y_pred_original))
print(classification_report(y_val_original, y_pred_original))

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  1  0  0]
 ...
 [ 0  0  0 ... 35  0  0]
 [ 0  0  0 ...  2  0  0]
 [ 0  0  0 ...  3  0  0]]
                       precision    recall  f1-score   support

           bookflight       0.00      0.00      0.00        42
          changeorder       0.00      0.00      0.00         5
 changeseatassignment       0.00      0.00      0.00        57
         checkbalance       1.00      0.62      0.77        50
     checkclaimstatus       0.14      0.01      0.02        90
checkoffereligibility       0.00      0.00      0.00         6
    checkserverstatus       0.00      0.00      0.00        30
         closeaccount       0.00      0.00      0.00        20
        disputecharge       0.18      0.12      0.14        42
        expensereport       0.00      0.00      0.00        77
      getboardingpass       1.00      0.03      0.05       114
 getinformationintent       0.00      0.00      0.00        32
        getpromotions   

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score

ari = adjusted_rand_score(y_val_original, y_pred_original)
ami = adjusted_mutual_info_score(y_val_original, y_pred_original, average_method='arithmetic')

print("ARI: {}".format(ari))
print("AMI: {}".format(ami))

ARI: 0.046535226129111484
AMI: 0.22931865195483248


## Predict on test data for submission

In [ ]:
X_test = df_test['message'].rename('text')

X_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)
df_test['numwords'] = df_test['message'].apply(lambda x: len(x.split()))
df_test['numnouns'] = df_test['message'].apply(count_pos, pos='NN')
df_test['numverbs'] = df_test['message'].apply(count_pos, pos='VB')
test_embeddings_metadata = np.concatenate((X_test_embeddings, np.array(df_test[['numwords', 'numnouns', 'numverbs']].values)), axis=1)

In [ ]:
predictions = lgbm_clf.predict(test_embeddings_metadata)

In [ ]:
transformed_predictions = labelEncoder.inverse_transform(predictions.astype('int32'))

In [ ]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': transformed_predictions})
print(my_submission.head())

      Id                 label
0  12123  reportbrokensoftware
1    244      orderpizzaintent
2   8221           replacecard
3  12856    startserviceintent
4  12108         expensereport


In [ ]:
compression_opts = dict(method='zip', archive_name='coda_submission.csv')
my_submission.to_csv('metadatahybrid_coda_submission.zip', index=False, compression=compression_opts)